In [1]:
# header files
import math, sys, time, glob, os
from openslide import open_slide, deepzoom
from PIL import ImageStat, Image
import numpy as np
print("Header files loaded...")

Header files loaded...


In [2]:
# function for extracting patches
def patch_extraction(wsi_path, output_path, tile_size=2000):
    # read slide
    slide = open_slide(wsi_path)
    
    # using deepzoom read non-empty regions
    dz = deepzoom.DeepZoomGenerator(slide, tile_size=tile_size, overlap=0, limit_bounds=True)
    mask_tile_size = tile_size*slide.level_downsamples[0] // slide.level_downsamples[0]
    dz_level = dz.level_count-1
    
    # get filename
    filename = wsi_path.split("/")[-1]
    filename = filename.split(".")[0]
    print(filename)
    
    # read entire slide
    mask = slide.read_region((0, 0), 0, slide.level_dimensions[0]).convert("L")
    fn = lambda x : 0 if x > 200 or x < 50 else 1
    mask = mask.point(fn, mode='1')
    
    # loop through each patch of the slide of size=(tile_size, tile_size)
    for i in range(dz.level_tiles[dz_level][0]):
        for j in range(dz.level_tiles[dz_level][1]):
            coord = dz.get_tile_coordinates(dz_level, (i, j))
            print(coord)
            if coord[2] != (tile_size, tile_size):
                continue
            else:
                coord = coord[0]
            cenX = (coord[0] + tile_size * slide.level_downsamples[0]//2) // slide.level_downsamples[0]
            cenY = (coord[1] + tile_size * slide.level_downsamples[0]//2) // slide.level_downsamples[0]
            mask_region = mask.crop((cenX-(mask_tile_size//2), cenY-(mask_tile_size//2), cenX+(mask_tile_size//2), cenY+(mask_tile_size//2)))
            if ImageStat.Stat(mask_region).mean[0] > 0.7:
                tile = dz.get_tile(dz_level, (i, j)).convert("RGB")
                tile_output_path = os.path.join(output_path, filename + "_" + str(coord[0]) + '_' + str(coord[1]) + '.png')
                tile.save(tile_output_path)

In [3]:
patch_extraction(wsi_path="../../sample_ovarian_cancer_data/TCGA-25-2401.svs", output_path="../../patches_ovarian_cancer_data/")

TCGA-25-2401
((0, 0), 0, (2000, 2000))
((0, 2000), 0, (2000, 2000))
((0, 4000), 0, (2000, 2000))
((0, 6000), 0, (2000, 2000))
((0, 8000), 0, (2000, 2000))
((0, 10000), 0, (2000, 2000))
((0, 12000), 0, (2000, 2000))
((0, 14000), 0, (2000, 2000))
((0, 16000), 0, (2000, 2000))
((0, 18000), 0, (2000, 2000))
((0, 20000), 0, (2000, 2000))
((0, 22000), 0, (2000, 2000))
((0, 24000), 0, (2000, 2000))
((0, 26000), 0, (2000, 2000))
((0, 28000), 0, (2000, 2000))
((0, 30000), 0, (2000, 2000))
((0, 32000), 0, (2000, 2000))
((0, 34000), 0, (2000, 2000))
((0, 36000), 0, (2000, 2000))
((0, 38000), 0, (2000, 2000))
((0, 40000), 0, (2000, 2000))
((0, 42000), 0, (2000, 2000))
((0, 44000), 0, (2000, 2000))
((0, 46000), 0, (2000, 2000))
((0, 48000), 0, (2000, 2000))
((0, 50000), 0, (2000, 2000))
((0, 52000), 0, (2000, 2000))
((0, 54000), 0, (2000, 2000))
((0, 56000), 0, (2000, 2000))
((0, 58000), 0, (2000, 2000))
((0, 60000), 0, (2000, 726))
((2000, 0), 0, (2000, 2000))
((2000, 2000), 0, (2000, 2000))
((200

((16000, 8000), 0, (2000, 2000))
((16000, 10000), 0, (2000, 2000))
((16000, 12000), 0, (2000, 2000))
((16000, 14000), 0, (2000, 2000))
((16000, 16000), 0, (2000, 2000))
((16000, 18000), 0, (2000, 2000))
((16000, 20000), 0, (2000, 2000))
((16000, 22000), 0, (2000, 2000))
((16000, 24000), 0, (2000, 2000))
((16000, 26000), 0, (2000, 2000))
((16000, 28000), 0, (2000, 2000))
((16000, 30000), 0, (2000, 2000))
((16000, 32000), 0, (2000, 2000))
((16000, 34000), 0, (2000, 2000))
((16000, 36000), 0, (2000, 2000))
((16000, 38000), 0, (2000, 2000))
((16000, 40000), 0, (2000, 2000))
((16000, 42000), 0, (2000, 2000))
((16000, 44000), 0, (2000, 2000))
((16000, 46000), 0, (2000, 2000))
((16000, 48000), 0, (2000, 2000))
((16000, 50000), 0, (2000, 2000))
((16000, 52000), 0, (2000, 2000))
((16000, 54000), 0, (2000, 2000))
((16000, 56000), 0, (2000, 2000))
((16000, 58000), 0, (2000, 2000))
((16000, 60000), 0, (2000, 726))
((18000, 0), 0, (2000, 2000))
((18000, 2000), 0, (2000, 2000))
((18000, 4000), 0, (2

((32000, 8000), 0, (2000, 2000))
((32000, 10000), 0, (2000, 2000))
((32000, 12000), 0, (2000, 2000))
((32000, 14000), 0, (2000, 2000))
((32000, 16000), 0, (2000, 2000))
((32000, 18000), 0, (2000, 2000))
((32000, 20000), 0, (2000, 2000))
((32000, 22000), 0, (2000, 2000))
((32000, 24000), 0, (2000, 2000))
((32000, 26000), 0, (2000, 2000))
((32000, 28000), 0, (2000, 2000))
((32000, 30000), 0, (2000, 2000))
((32000, 32000), 0, (2000, 2000))
((32000, 34000), 0, (2000, 2000))
((32000, 36000), 0, (2000, 2000))
((32000, 38000), 0, (2000, 2000))
((32000, 40000), 0, (2000, 2000))
((32000, 42000), 0, (2000, 2000))
((32000, 44000), 0, (2000, 2000))
((32000, 46000), 0, (2000, 2000))
((32000, 48000), 0, (2000, 2000))
((32000, 50000), 0, (2000, 2000))
((32000, 52000), 0, (2000, 2000))
((32000, 54000), 0, (2000, 2000))
((32000, 56000), 0, (2000, 2000))
((32000, 58000), 0, (2000, 2000))
((32000, 60000), 0, (2000, 726))
((34000, 0), 0, (2000, 2000))
((34000, 2000), 0, (2000, 2000))
((34000, 4000), 0, (2

KeyboardInterrupt: 